# Chapter 15: OpenCV

**OpenCV** provides many image processing and computer vision tools.  While written in C++, it provides a fairly extensive Python interface, enabling it to be used as a Python toolbox, see: [https://docs.opencv.org/4.5.3/](https://docs.opencv.org/4.5.3/).  Use OpenCV for the many powerful image operations such as segmentation, calibration, warping, and even applying deep neural networks to images.  Plotting is fairly limited, and more powerful plotting is available in Matplotlib, see  [Chapter 16: Matplotlib](Chapter_16_Matplotlib.ipynb).

For installation, see [Chapter 13](Chapter_13_Numpy.ipynb).

## Image Operations with OpenCV

OpenCV reads images into Numpy arrays:

In [1]:
import cv2 as cv
import numpy as np
img = cv.imread('.Images/book.png', -1)

Here I have provided the full path from where Python is running to the `book.png` image in this repo.  The `-1` flag reads in raw data without changing the format or number of channels.  You'll need to adjust your path to point to this image from where you are running Python.  Now let's examine the image we just read in:

In [2]:
type(img)

numpy.ndarray

If you see an output `NoneType`, then `img` was not successfully loaded.  You'll need to adjust the path to the image.  If you want to know the path of where Python is currently running use this:

In [3]:
import os
os.getcwd() 

'c:\\Users\\morri\\Source\\Repos\\av\\python_intro\\Docs'

I assume you have adjusted the path and successfully read in the image, and it is a `numpy.ndarray`.  Let's examine its size and the pixel type:

In [4]:
img.shape

(247, 160, 3)

We see that it is a 3D array; the `.shape` is a tuple with array dimension sizes, in this case: (height, width, nchannels).  We can think of an image as three single channel arrays each size (height, width) that are stacked along the third dimension.  The three channels are (blue, green, red), as this is the default format for OpenCV, which differs from the more usual (red, green, blue) format.  Notice the data type:

In [5]:
img.dtype 

dtype('uint8')

The data type is an unsigned 8-bit type (`uint8`) which stores numbers from 0 to 255 inclusive.  We can extract the 3 components of a sample single pixel as follows:

In [6]:
img[22,28,:]

array([212, 187, 143], dtype=uint8)

Or we can drop the trailing colon to get the same output:

In [7]:
img[22,28]  

array([212, 187, 143], dtype=uint8)

These three values are the (blue, green, red) values for that pixel.  Review Numpy [indexing](https://numpy.org/doc/stable/reference/arrays.indexing.html) if this isn't clear.

Let's say our goal is to find the horizontal image gradient of the grayscale version of this image.  Here are the steps we would take.

In [8]:
imgf = img / 255 

This converts to a floating point array and scales to a range of 0 to 1, which is the usual range for floating point images.  It is best to use floating point rather than unsigned integers because the gradient can be negative or could be larger than the maximum `uint8` value of 255.  Operating in `uint8` will result in gradients being clipped.  Confirming our new data type:

In [9]:
imgf.dtype

dtype('float64')

Next, convert the image to grayscale with `cvtColor`, which is a general purpose colorspace transformation utility:

In [10]:
imgf_gray = cv.cvtColor(imgf.astype(np.float32), cv.COLOR_BGR2GRAY)  # Convert 3-channel color to single-channel grayscale

If you do `help(cv.cvtColor)` you will see that `cvtColor` requires floating-bit arrays be 32-bit, while `img` is 64-bit, so we passed in a 32-bit version of `img`.  This grayscale image is the same size, except with a single channel:

In [11]:
imgf_gray.shape

(247, 160)

Finally, let's apply a Sobel operator to obtain the image gradient in x:

In [12]:
imgf_grad_x = cv.Sobel(imgf_gray, cv.CV_32F, 1, 0, ksize=5)

The `cv.CV_32F` argument says that the provided image is a 32-bit floating point array. To see the meaning of the remaining arguments use: `help(cv.Sobel)`.  

Next we'll show how to plot images with OpenCV, but save plotting the gradient image for Matplotlib in [Chapter 16: Matplotlib](Chapter_16_Matplotlib.ipynb) which has more powerful plotting capabilities.

## Plotting with OpenCV
OpenCV has some graphical display functions including showing images.  For instance:

In [13]:
cv.imshow('Image',img)
cv.waitKey(1)            # This gives code 1 millisecond to actually draw the window and then move on

-1

This will show an image in a **separate window** that should look like this:

![Image](.Images/show_book.png)

Find the window (which might be underneath this document window).  Note, that the window is not being refreshed.  If you want it to be refreshed, you can instead call it no arguments: `cv.waitKey()`, and this will stop code progression until you close the window.  To clear any drawn windows, use:    

In [14]:
cv.destroyAllWindows()

Let's say we want to draw a circle and rectangle at the image center.  The OpenCV `circle` and `rectangle` functions will actually change the pixel values, so if we want to avoid changing our `img` variable we could make a copy of it just for plotting as follows:

In [15]:
img_cp = img.copy()
img_cen_xy = (img_cp.shape[1]//2, img_cp.shape[0]//2)  # Integer division and order (x,y) 
rectangle = img_cen_xy + (40,20)                # (width=40,height=20):  Concatenates 2 tuples to make bounding box: (left,top,width,height) 
cv.circle( img_cp, img_cen_xy, radius=15, color=(0,0,255), thickness=2 )
cv.rectangle( img_cp, rectangle, color=(0,255,0), thickness=2 )
cv.imshow('Image with Circle', img_cp)
cv.waitKey(1)                                # This gives code 1 millisecond to draw the window and then move on

-1

Again, find this window to see the output which should look like this:

![Image](.Images/book_circle.png)

Notice the pixelation as the circle is part of the image.  Also, notice that `cv.waitKey(1)` is called with a 1-millisecond argument which is sufficient for OpenCV to draw the window and then move on.  

## Masks
A common image processing tool is an image mask.  Typically a mask will have size equal in height and width to an image and have a single value per pixel. Let's create a mask that is true for all the pixels whose red component is greater than its blue component: 

In [16]:
mask = img[:,:,2] > img[:,:,0]
cv.imshow('Mask of red > blue',mask.astype(np.uint8)*255)
cv.waitKey(1)

-1

![Mask](.Images/mask.png)

Here the red channel is `img[:,:,2]` and blue is `img[:,:,0]`.  Its type is `bool`; that is, each value is `True` or `False`.  Notice that to display the mask I converted it to an unsigned 8-bit image and multiplied by 255 so that all values are 0 (black) or 255 (white).  We can see that the mask shape is the same as the image, except with a single channel:

In [17]:
mask.shape

(247, 160)

Masks can be combined using Numpy's logical operations, such as `logical_and`, or alternatively elementwise multiplication of boolean arrays is the same thing as a logical `AND`. Let's say we want a mask where red values are greater than blue and red values are greater than 128.  This can be done with: 

In [18]:
new_mask = np.logical_and( img[:,:,2] > img[:,:,0], img[:,:,2] > 128 )
cv.imshow('Mask of red > blue',new_mask.astype(np.uint8)*255)
cv.waitKey(1)

-1

![New Mask](.Images/new_mask.png)

We could apply the mask to the image, namely zero out all the pixels that are `False` in the mask.  Multiplication by the mask and leveraging Numpy broadcasting will do that easily for us.  Note that we first need to convert the mask into a 3D array to match the number of dimensions in the color image.  We can add a single third channel using `mask[:,:,None]`:

In [19]:
masked_img = img * new_mask[:,:,None]
cv.imshow('Masked Image',masked_img)
cv.waitKey(1)

-1

![Masked Image](.Images/Masked_Image.png)

In order to delete the above windows call:

In [20]:
cv.destroyAllWindows()

## Connected Components
The OpenCV function: `cv.connectedComponentsWithStats` is quite useful.  You can find detailed information on its arguments using `help(cv.connectedComponentsWithStats)`.  The following is an example of its use.  First define an 8-bit image with zeros as the background and non-zeros as the components:

In [21]:
nimg = ( np.array([1,1,0,1,1,1])[:,None] * np.array([1,1,0,1,1,1,1,1]) ).astype(np.uint8)
nimg

array([[1, 1, 0, 1, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1, 1]], dtype=uint8)

Now find the connected components along with details for each component as follows:

In [22]:
N, label_img, bbn, centroids = cv.connectedComponentsWithStats( nimg )
print('Image with each pixel assigned a label value:')
print(label_img)
print('Label, Bounding Box,  Npix,  Centroid')
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})   # For nice output formatting
for i in range(N):
    print(f'{i:3}      {bbn[i,:4]}   {bbn[i,4]:4}    {centroids[i,:]}')

Image with each pixel assigned a label value:
[[1 1 0 2 2 2 2 2]
 [1 1 0 2 2 2 2 2]
 [0 0 0 0 0 0 0 0]
 [3 3 0 4 4 4 4 4]
 [3 3 0 4 4 4 4 4]
 [3 3 0 4 4 4 4 4]]
Label, Bounding Box,  Npix,  Centroid
  0      [0 0 8 6]     13    [ 2.923  2.231]
  1      [0 0 2 2]      4    [ 0.500  0.500]
  2      [3 0 5 2]     10    [ 5.000  0.500]
  3      [0 3 2 3]      6    [ 0.500  4.000]
  4      [3 3 5 3]     15    [ 5.000  4.000]


The input and output arguments for this line are given below:
```python
N, label_img, bbn, centroids = cv.connectedComponentsWithStats( nimg )
```
```
nimg:       [HxW] Input image in uint8 format with 0 as background pixels
N:          Number of discovered connected components
label_img:  [HxW] Each pixel is given its connected component label
bbn:        [Nx5] Each row: [left,top,width,height,Npix] is bounding box and
                  number of pixels in the connected component
centroids:  [Nx2] Each row is the centroid for the component
```

The label values in `label_img` are `[0,...,N-1]` with 0 being the background label.  Notice the connected component with label 0.  I think the background label is always the first row of `bbn` and `centroids`.  

___
### [Outline](../README.md), Next: [Chapter 16: Matplotlib](Chapter_16_Matplotlib.ipynb)